In [1]:
from local.node import Node
from local.constnode import ConstNode
from local.varnode import VarNode
from local.funcnode import FuncNode
from local.func import *
from local.treeoptimizer import TreeOptimizer
from local.crossover import *
from local.Optimizer import *

import os
import json
import glob
import copy
import random
import pickle
import graphviz
import datetime
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, metrics, optimizers

In [2]:
def create_folder():
    date = datetime.datetime.today()
    folder_path = str(date.year)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

    folder_path += "/"+str(date.month) +"_"+str(date.day)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

    folder_path += "/"+str(date.hour)+"_"+str(date.minute)
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        
    return folder_path


def Tounament_select(score_list, generation, Selection_size):
    candidate = random.sample(range(len(score_list)), Selection_size)
    candidate_score = [score_list[i] for i in candidate]
    index = candidate_score.index(max(candidate_score))
    return copy.deepcopy(generation[candidate[index]])

In [3]:
Population = 50
Max_generation = 500

Mutation_rate = 0.2
Crossover_rate = 0.8

Selection_size = 2

BATCH_SIZE = 512
EPOCHS = 10

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

generation = []
generation += [Momentum() for _ in range(Population // 2)]
generation += [RMSProp() for _ in range(Population // 2)]

best_optim = SGD()
best_score = 0.0
best_score_list = []
past_progress = 0
folder_path = create_folder()
if not os.path.exists(folder_path+"/best_optim/"):
    os.mkdir(folder_path+"/best_optim/")
score_progress = []

In [4]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train, x_test = x_train[..., tf.newaxis] / 255.0, x_test[..., tf.newaxis] / 255.0

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

    return train_dataset, test_dataset

def build_model():
    input = layers.Input(shape = (32, 32, 3))
    x = layers.Conv2D(32, 3, 2)(input)
    x = layers.LeakyReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units = 128, activation = "relu")(x)
    output = layers.Dense(units = 10, activation = "softmax")(x)

    return models.Model(input, output)

train_dataset, test_dataset = load_data()

model = build_model()
model.compile()
if not os.path.exists("models/"):
    os.mkdir("models/")
if not os.path.exists("models/model_cifar.keras"):
    model.save_weights("models/model_cifar.keras")

In [11]:
for progress in range(past_progress, Max_generation):
    fitness_list = []
    save_path = folder_path + f"/generation{progress}/"
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    for index in tqdm(range(len(generation))):

        acc_list = []
        optimizer = generation[index]
        model.load_weights("models/model_cifar.keras")

        @tf.function
        def train_step(X, Y):
            with tf.GradientTape() as tape:
                pred = model(X)
                loss = losses.SparseCategoricalCrossentropy()(Y, pred)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
            loss_obj(loss)
            accuracy_obj(Y, pred)


        @tf.function
        def test_step(X, Y):
            pred = model(X)
            
            val_loss_obj(Y, pred)
            val_accuracy_obj(Y, pred)

        for i in range(EPOCHS):
            
            loss_obj.reset_state()
            accuracy_obj.reset_state()
            val_loss_obj.reset_state()
            val_accuracy_obj.reset_state()
            for X, Y in train_dataset:
                train_step(X, Y)
            
            for X, Y in test_dataset:
                test_step(X, Y)
        
            acc_list.append(val_accuracy_obj.result().numpy())
        fitness_list.append(max(acc_list))
        optimizer.plot_struct(f"{save_path}/optimizer_{index}_{max(acc_list)}")
        pickle_file = open(f"{save_path}/optimizer_{index}_{max(acc_list)}.pickle", mode= "wb")
        pickle.dump(optimizer, pickle_file)
        pickle_file.close()


    for index in range(len(fitness_list)):
        if fitness_list[index] > best_score:
            best_score = fitness_list[index]
            best_optim = copy.deepcopy(generation[index])

            best_optim.plot_struct(f"{folder_path}/best_optim/generation_{progress}_{index}_{best_score}")
            pickle_file = open(f"{folder_path}/best_optim/generation_{progress}_{index}_{best_score}.pickle", mode= "wb")
            pickle.dump(best_optim, pickle_file)
            pickle_file.close()
    
    best_score_list.append(best_score)
    print(best_score)
    json_file = open(f"{folder_path}/best_optim/score{progress}.json", "w")
    json.dump(str(best_score), json_file)
    json_file.close()


    next_generation = []

    while len(next_generation) < Population:
        parent1 = Tounament_select(fitness_list, generation, Selection_size)
        parent2 = Tounament_select(fitness_list, generation, Selection_size)

        if random.random() < Crossover_rate:
            parent1, parent2 = crossover(parent1, parent2)
        
        next_generation.append(parent1)
        next_generation.append(parent2)
    
    for index, genetic in enumerate(next_generation):
        if random.random() < Mutation_rate:
            genetic.mutate()

    next_generation.append(copy.deepcopy(best_optim))
    next_generation.append(Momentum())
    next_generation.append(RMSProp())        
    
    generation = next_generation[len(next_generation) - Population:]

    # pickleファイルが思ったより容量大きかったので、現世代の評価と進化が終わったら、ひとつ前の世代のpickleファイルを削除
    check_path = folder_path + f"/generation{progress -1}"
    pickle_list = glob.glob(check_path +"/*.pickle")
    for path in pickle_list:
        os.remove(path)

 70%|███████   | 35/50 [14:10<06:02, 24.17s/it]dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.873414 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.881212 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.883208 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.886337 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.888765 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.898317 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.904466 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.910878 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.917817 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.924864 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.932019 to fit
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.939286 to fit
dot: 

In [ ]:
past_progress = 42

read_folder_path = folder_path +f"/{past_progress}"
pickle_list = glob.glob(read_folder_path +"/*.pickle")
tmp_generation = []
for path in pickle_list:
    genetic = pickle.load(path)
    tmp_generation.append(genetic)